<a href="https://colab.research.google.com/github/tugangui/caseTracker/blob/main/caseTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# List the tab names. Could be used for asserting in next section
# titles_list = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheets()
# print(titles_list)

Mounted at /content/drive


In [ ]:
# tab names and pivotal column (case column)
tab_names = {'U01 ':1,'U19 Salk Institute':1,'U19 CSHL':1,'MCP':1,'Basal Ganglia':1,'RF1 (HPF)':1,'BLA project':1}
tabs = {}
for k,v in tab_names.items():
  worksheet = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheet(k)
  tabs[k] = pd.DataFrame(worksheet.get_all_values())  

distinctCases = pd.DataFrame()
tracersPathways = pd.DataFrame()
actualInjections = pd.DataFrame()

# ToDo: Please revist the parsing
for k,tab in tabs.items():  
  piv = tab_names[k]
  cases = pd.DataFrame(tab.iloc[:, piv].unique()) # Distinct cases  
  cases = cases.applymap(str)
  cases = cases.loc[cases[0].str.startswith('S')]
  cases['length']= (cases[0]).str.len()
  cases = cases[(cases.length>=11) & (cases.length<=13)] # some cases have asterisks?


  tracers = pd.DataFrame(tab.iloc[:, piv+2]) # total
  mapping = {tracers.columns[0]:'tracers'}
  tracers = tracers.rename(columns=mapping)
  tracers['length']= (tracers['tracers']).str.len()
  tracers = tracers[(tracers.length>1)] # filter out blank entries

  if(k=='BLA project' or k=='U01 '):
    injections = pd.DataFrame(tab.iloc[:, piv+4]) # Intended injections column is column 5
  else:
    injections = pd.DataFrame(tab.iloc[:, piv+3]) # total
  mapping = {injections.columns[0]:'injections'}
  injections = injections.rename(columns=mapping)
  injections['length']= (injections['injections']).str.len()
  injections = injections[(injections.length>1)] # filter out blank entries

  distinctCases = distinctCases.append(cases)
  tracersPathways = tracersPathways.append(tracers)
  actualInjections = actualInjections.append(injections)

tracersPathways = tracersPathways.dropna()
actualInjections = actualInjections.dropna()
distinctCases = distinctCases.dropna()

distinctCases = pd.DataFrame(distinctCases.iloc[:,0].unique())
distinctCases.index = distinctCases.index + 1
distinctCases.to_csv('/distinctCases.csv', header=False)
print('Distinct cases: '+ str(distinctCases.size))
print('/distinctCases.csv')

tracersPathways = tracersPathways.iloc[:, 0]
tracersPathways.index = tracersPathways.index + 1
pd.DataFrame(tracersPathways).to_csv('/tracersPathways.csv', header=False)
print('Total tracers: '+ str(tracersPathways.size))
print('/tracersPathways.csv')

actualInjections = actualInjections.iloc[:, 0]
actualInjections.index = actualInjections.index + 1
pd.DataFrame(actualInjections).to_csv('/actualInjections.csv', header=False)
print('Total injections: ' + str(actualInjections.size))
print('/actualInjections.csv')

ser = [distinctCases, tracersPathways, actualInjections]
my_names = ['Cases','Injections','Tracers']

df = pd.concat(ser, axis=0, keys=my_names)
df.to_csv('/allColumns.csv')
print('/allColumns.csv')

Distinct cases: 1788
/distinctCases.csv
Total tracers: 7523
/tracersPathways.csv
Total injections: 6468
/actualInjections.csv
/allColumns.csv
